In [1]:
#
#  Tzerefos Anargiros - 2022202004022 - dit2022dsc@office365.uop.gr
#  Deligiannis Panagiotis - 2022202004004 - dit2004dsc@office365.uop.gr
#
# IMPORT REQUIRED DEPENDENCIES
import pandas as pd
from matplotlib import pyplot as plt
import pymongo
from pymongo import MongoClient
import sys
import json
import math

In [85]:
# THE INPUT FILE ("netflix_titles.csv") PATH
input_file_dir = "../data/input/netflix_titles.csv" 

# THE OUTPUT PATH FOR THE PRODUCED JSON FILE 
json_output_path = "../data/output/netflix_titles.json" 

# IMPORT INPUT FILE INTO A DATAFRAME
df = pd.read_csv(input_file_dir)

In [86]:
# DISPLAY DF
display(df.head())
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [87]:
# DROP DUPLICATES WHICH SPAN IN ALL COLUMNS
df = df.drop_duplicates()
display(df[df.duplicated(['show_id'])])

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


In [88]:
for index, row in df.iterrows():
    
    if not isinstance(row['date_added'], float) and row['date_added'] != '': # CAPTURE NANs
        if not isinstance(row['cast'], float) and "Tremaine Brown,  Jr." in row["cast"]:
            #display( df[df['show_id'] == row['show_id']] )
            #print(row['cast'])
            df.at[index,'cast'] = row['cast'].replace("Tremaine Brown,  Jr.", "Tremaine Brown Jr.")
        month_day = row['date_added'].lstrip().split(", ") # REMOVE LEADING WHITESPACE
        year = month_day[1]
        month = month_day[0].split(" ")[0]
        day = month_day[0].split(" ")[1]
        new_date = str(day)+"-"+str(months.index(month.lower()))+"-"+str(year)
        df.at[index,'date_added'] = new_date
            
        
df['date_added'] = pd.to_datetime(df['date_added'], format='%d-%m-%Y', errors='ignore')

In [89]:
# OUTPUT DATAFRAME INTO JSON FILE
df.to_json(json_output_path, orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)